# Análise resultados Jusbrasil

https://carguero.atlassian.net/browse/DADOS-294

### Libraries

In [108]:
import pandas as pd
from unidecode import unidecode

### Data ingestion

In [109]:
df_cpfs = pd.read_csv('data/amostra_300.csv')

df_cpfs['cod_cpf'] = df_cpfs['cod_cpf'].astype(str).str.zfill(11)

In [110]:
df_escavador = pd.read_csv('data/results_escavador.csv')

df_escavador = df_escavador[['cod_cpf', 'result_escavador_manual']]
df_escavador['cod_cpf'] = df_escavador['cod_cpf'].astype(str).str.zfill(11)
df_escavador['result'] = df_escavador['result_escavador_manual'].apply(lambda x: 1 if x == 'CRIMINAL' else 0)

In [111]:
df_jusbrasil = pd.read_excel('drivers_lawsuits_details_300.xlsx')

df_jusbrasil['matchRelatedPeople_name_UPPER'] = df_jusbrasil['matchRelatedPeople_name'].str.upper().apply(unidecode)
df_jusbrasil = df_jusbrasil.merge(df_cpfs.drop_duplicates(subset=['nm_motorista'])[['nm_motorista', 'cod_cpf', 'ROUBO']],
                                  left_on='matchRelatedPeople_name_UPPER', right_on='nm_motorista', how='left')

### Motoristas com processo encontrados pelo Escavador, porém não encontrados pela Jusbrasil.

In [112]:
mot_set = set(df_escavador[df_escavador['result_escavador_manual']!='NÃO ENCONTRADO']['cod_cpf']) - set(df_jusbrasil[df_jusbrasil['ROUBO']==1]['cod_cpf'].values)
len(mot_set), mot_set

(29,
 {'00096913592',
  '00632083182',
  '00902957066',
  '01301501182',
  '02387430123',
  '02428273906',
  '04063153100',
  '04294169974',
  '07097699469',
  '15030504915',
  '17112010144',
  '21482348896',
  '26200575827',
  '31467636053',
  '33003625802',
  '34199155864',
  '36452356801',
  '47522828104',
  '51119765072',
  '51460858115',
  '57156417134',
  '65533151915',
  '81164700197',
  '82753946868',
  '87037459100',
  '90693353104',
  '91998344053',
  '95800930830',
  '98037838153'})

### Motoristas com processo encontrados pela Jusbrasil, porém não encontrados pelo Escavador.

In [113]:
mot_set = set(df_jusbrasil[df_jusbrasil['ROUBO']==1]['cod_cpf'].values) - set(df_escavador[df_escavador['result_escavador_manual']!='NÃO ENCONTRADO']['cod_cpf'])
len(mot_set)

1

### Acurácia - Jusbrasil

In [117]:
df_jusbrasil.drop_duplicates(subset=['matchRelatedPeople_name']).shape

(55, 68)

In [118]:
print(f"{17/50*100.:2}% motoristas com roubo tiveram processos criminais encontrados.")

34.0% motoristas com roubo tiveram processos criminais encontrados.


In [119]:
print(f"{(55-17)/250*100.:2}% motoristas sem roubo tiveram processos criminais encontrados.")

15.2% motoristas sem roubo tiveram processos criminais encontrados.


Alguns motoristas não dão match com nome encontrado no processo e o nome da base de dados.

### Conclusão

* Jusbrasil apresentou acurácia de 34% dos 50 motoristas com roubo e 15.2% de falsos positivos.
* A solução, já possui uma regra para identificar processos criminais, sendo assim, é mais simples do que o Escavador, porém
apresenta menor complexidade.
* Solução tem potencial para agregar a solução de análise de risco Carguero.